In [3]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import talib as tb
import yfinance as yf

import pickle


import datetime
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import *
from datetime import timedelta

In [4]:
def df_converter(df): 
    df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    df_dollar = yf.download('DX=F',  
                      progress=False)
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    # MA df
    df_ma = df['Close'].to_frame()
    df_ma['SMA30'] = df_ma['Close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['Close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['Close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["Close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['Close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['Close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['High'], df['Low'], df['Close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['Close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.fillna(method='ffill', inplace=True)
    result.dropna(inplace=True)

    return result 

In [5]:
df_coin = yf.download('BTC-USD',
                      end=date.today(), 
                      progress=False,
    )


In [6]:
df_coin

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2022-05-18,30424.478516,30618.716797,28720.271484,28720.271484,28720.271484,31285268319
2022-05-19,28720.359375,30430.751953,28708.955078,30314.333984,30314.333984,33773447707
2022-05-20,30311.119141,30664.976562,28793.605469,29200.740234,29200.740234,30749382605


In [7]:
df_coin = df_converter(df_coin)

In [8]:
df_coin.index

DatetimeIndex(['2017-11-09', '2017-11-10', '2017-11-11', '2017-11-12',
               '2017-11-13', '2017-11-14', '2017-11-15', '2017-11-16',
               '2017-11-17', '2017-11-18',
               ...
               '2022-05-14', '2022-05-15', '2022-05-16', '2022-05-17',
               '2022-05-18', '2022-05-19', '2022-05-20', '2022-05-21',
               '2022-05-22', '2022-05-23'],
              dtype='datetime64[ns]', name='Date', length=1657, freq=None)

In [9]:
start_date = '2021-10-22'

In [10]:
start_date = pd.to_datetime(start_date)

In [11]:
from datetime import datetime


In [12]:
start_date

Timestamp('2021-10-22 00:00:00')

In [13]:
df_coin

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3.226250e+09,6720.195964,1012.489040,6798.973215,37.428679,...,7929.899976,547.298468,521.809901,25.488567,65.836431,604.852006,33.184032,38.772059,94.351997,2584.620117
2017-11-10,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5.208250e+09,6767.749967,1017.359285,6776.369081,36.238180,...,7899.045668,482.149729,513.877867,-31.728137,55.203600,565.647850,30.022892,33.577955,94.278000,2582.300049
2017-11-11,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4.908680e+09,6806.196647,1021.994910,6724.022958,34.548539,...,7835.311479,404.828866,492.068066,-87.239201,50.841315,514.557416,13.668599,25.625174,94.278000,2582.300049
2017-11-12,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8.957350e+09,6819.328646,1026.269043,6627.278816,32.896648,...,7798.154128,307.126870,455.079827,-147.952957,44.895661,489.412741,14.127749,19.273080,94.278000,2582.300049
2017-11-13,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6.263250e+09,6846.371322,1031.063862,6618.805243,30.978407,...,7771.077715,275.694434,419.202748,-143.508315,53.591318,462.353197,27.607649,18.467999,94.388000,2584.840088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-19,28720.359375,30430.751953,28708.955078,30314.333984,30314.333984,3.377345e+10,31432.940755,12219.976696,31714.919811,41.932405,...,36674.094728,-2639.569025,-2602.047302,-37.521722,37.867254,2620.576987,42.854248,54.520355,102.750999,4123.339844
2022-05-20,30311.119141,30664.976562,28793.605469,29200.740234,29200.740234,3.074938e+10,30941.314063,12226.034763,31400.647364,42.318692,...,35499.863648,-2595.389238,-2600.715690,5.326451,35.400117,2279.274793,27.123670,38.620103,103.172997,4123.339844
2022-05-21,29199.859375,29588.869141,29027.394531,29432.226562,29432.226562,1.727484e+10,30500.734375,12232.171066,31154.594764,42.679227,...,34198.881372,-2512.732215,-2583.118995,70.386780,36.324105,1849.073499,39.043883,36.340600,103.172997,4123.339844


In [14]:
df_coin = df_coin[df_coin.index >= start_date]

In [15]:
df_coin.head()

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-22,62237.890625,63715.023438,60122.796875,60692.265625,60692.265625,3.843408e+10,59411.947135,9548.177219,59279.606622,42.268428,...,66512.766057,4029.816080,3784.073284,245.742796,61.311059,3550.409461,45.656328,70.563090,93.625000,4544.899902
2021-10-23,60694.628906,61743.878906,59826.523438,61393.617188,61393.617188,2.688255e+10,59906.998437,9568.163896,59543.857943,41.723052,...,66432.717203,3820.417900,3791.342207,29.075693,62.480777,3262.859383,23.700462,48.104014,93.625000,4544.899902
2021-10-24,61368.343750,61505.804688,59643.343750,60930.835938,60930.835938,2.731618e+10,60304.505990,9587.956833,59717.230192,41.121085,...,66281.803561,3575.904948,3748.254755,-172.349808,61.173225,2988.648786,16.516932,28.624574,93.625000,4544.899902
2021-10-25,60893.925781,63729.324219,60691.800781,63039.824219,63039.824219,3.106491e+10,60855.722396,9608.546920,60132.554446,41.093764,...,66179.665938,3511.822299,3700.968264,-189.145965,64.772807,2661.971771,29.515861,23.244418,93.808998,4566.479980
2021-10-26,63032.761719,63229.027344,59991.160156,60363.792969,60363.792969,3.487897e+10,61047.655990,9628.090719,60161.459261,40.705583,...,66070.739534,3208.121750,3602.398961,-394.277211,57.522762,2511.541772,28.040026,24.690939,93.941002,4574.790039


# Predictions Graph

In [16]:
def add_days(df, forecast_length):
    end_point = len(df)
    df1 = pd.DataFrame(index=range(forecast_length), columns=range(2))
    df1.columns = ['Close', 'Date']
    df = df.append(df1)
    df = df.reset_index(drop=True)
    x = df.at[end_point - 1, 'Date']
    x = pd.to_datetime(x, format='%Y-%m-%d')
    for i in range(forecast_length):
        df.at[df.index[end_point + i], 'Date'] = x + timedelta(days=1+i)
        df.at[df.index[end_point + i], 'Close'] = 0
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df = df.drop(['Date'], axis=1)
    return df

In [17]:
def forecasting(model,df1, forecast_length):
    df3 = df1[['Close', 'Date']]
    df3 = add_days(df3, forecast_length)
    finaldf = df1.drop('Date', axis=1)
    finaldf = finaldf.reset_index(drop=True)
    end_point = len(finaldf)
    x = end_point - forecast_length
    finaldf_train = finaldf.loc[:x - 1, :]
    finaldf_train_x = finaldf_train.loc[:, finaldf_train.columns != 'Close']
    finaldf_train_y = finaldf_train['Close']

    fit = model.fit(finaldf_train_x, finaldf_train_y)
    yhat = []
    end_point = len(finaldf)
    df3_end = len(df3)
    for i in range(forecast_length, 0, -1):
        y = end_point - i
        inputfile = finaldf.loc[y:end_point, :]
        inputfile_x = inputfile.loc[:, inputfile.columns != 'Close']
        pred_set = inputfile_x.head(1)
        pred = fit.predict(pred_set)
        df3.at[df3.index[df3_end - i], 'Close'] = pred[0]
        finaldf = df1.drop('Date', axis=1)
        finaldf = finaldf.reset_index(drop=True)
        yhat.append(pred)
    yhat = np.array(yhat)
    return yhat

In [18]:
def predictions(df_coin,model, forecast_lenght = 5, train_lenght = 100,target = 'Close'):
    """ df_coin must be with date in index,
        forecast_lenght is the amount of days that we will predict
        model is the model predefined to use to get our predictions
        train_length is the amount of days that we will use to train the model
        target is what we are predicting
        This will return a graphic that will contain the data from the train set and our predictions
    """
    df_coin = df_coin.tail(train_lenght)
    df_coin.reset_index(inplace=True)
    df_coin['Date'] = pd.to_datetime(df_coin['Date'], format='%Y-%m-%d')

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=df_coin['Date'], y=df_coin[target], 
                    name='Actual Values', mode='lines',line=dict(color='black')))
    #Predictions
    forecast = forecasting(model,df_coin,forecast_lenght)
    #df that will contain the predictions
    df_pred = pd.DataFrame(columns=['Date',target])
    #Adding the predictions to our dataset
    for day, x in enumerate(forecast):
        new_row={'Date':df_coin['Date'].max() + timedelta(days=1+day),
     target:x[0]}
        df_pred = df_pred.append(new_row, ignore_index=True)

    df_pred['Date'] = pd.to_datetime(df_pred['Date'], format='%Y-%m-%d')

    fig2.add_trace(go.Scatter(x=df_pred['Date'], y=df_pred[target], name='Predictions', mode='lines',line=dict(color='red')))
    fig2.update_layout(dict(updatemenus=[
                        dict(
                        type = "buttons",
                        direction = "left",
                        buttons=list([
                                dict(
                                args=["visible", "legendonly"],
                                label="Deselect All",
                                method="restyle"
                                ),
                                dict(
                                args=["visible", True],
                                label="Select All",
                                method="restyle"
                                )
                        ]),
                        pad={"r": 10, "t": 10},
                        showactive=False,
                        x=1,
                        xanchor="right",
                        y=1.1,
                        yanchor="top"
                        ),
                ]
        ))
    fig2.show()

In [19]:
from sklearn.ensemble import RandomForestRegressor
rf_final = RandomForestRegressor(random_state=10,criterion='mae', max_depth=20, max_features='auto', n_estimators=30)

In [20]:
predictions(df_coin, rf_final, 5, 20, 'Close')

C:\Users\Admin\AppData\Local\Temp/ipykernel_22504/2344453812.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coin['Date'] = pd.to_datetime(df_coin['Date'], format='%Y-%m-%d')
c:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\ensemble\_forest.py:395: FutureWarning:

Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.



In [21]:
import finance_lib as fb

In [22]:
fb.predictions(df_coin, rf_final, 5, 100, 'Close')

c:\Users\Admin\Desktop\NOVAIMS\BC\BC5\finance_lib.py:147: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\ensemble\_forest.py:395: FutureWarning:

Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.



## Market Info

### Days Variation

In [23]:
df_coin = yf.download('BTC-USD',
                      end=date.today(), 
                      progress=False,
    )

In [24]:
df_coin = df_converter(df_coin)
day_before = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0))
monthly = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0))
yearly = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0))


In [25]:
today = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=0))
day_before = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=1))
monthly = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=30))
yearly = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=365))

In [26]:
day_before_value = df_coin[df_coin.index == day_before]['Close'][0]

In [27]:
df_coin[df_coin.index == today]['Close']

Date
2022-05-23    30323.722656
Name: Close, dtype: float64

In [28]:
import plotly.graph_objects as go

today = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=0))
day_before = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=1))
monthly = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=30))
yearly = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=365))

daily_percentage = go.Figure()
monthly_percentage = go.Figure()
yearly_percentage = go.Figure()

if df_coin[df_coin.index == today]['Close'].empty:
    today_value = df_coin[df_coin.index == day_before]['Close'][0]
    day_before = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=2))
    day_before_value = df_coin[df_coin.index == day_before]['Close'][0]
    
else: 
    today_value = df_coin[df_coin.index == today]['Close'][0]
    day_before_value = df_coin[df_coin.index == day_before]['Close'][0]

monthly_value = df_coin[df_coin.index == monthly]['Close'][0]
yearly_value = df_coin[df_coin.index == yearly]['Close'][0]



In [97]:
def get_percentage_img(current_value, prev_value):
    fig = go.Figure()
    
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = current_value,
        delta = {'reference': prev_value, 'relative': True, 'valueformat':'.2%'},
        domain = {'x': [0, 1], 'y': [0, 1]}))
    fig.update_layout(
            font_color="black",
            font_size= 15,
            height=50,margin={'t': 0,'l':0,'b':10,'r':0}
        )

    fig.show()

In [98]:
get_percentage_img(today_value,monthly_value)

### Open Close Volume

In [99]:
df_coin = yf.download('BTC-USD',
                      end=date.today(), 
                      progress=False,
    )
df_coin_day = yf.download('BTC-USD',
                      start=date.today(), 
                      interval="1m",
                      progress=False,
    )

In [100]:
# crypto first day in dataset
crypto_first_day = str(df_coin.index.min())
crypto_first_day = crypto_first_day[:10]
crypto_first_day

'2014-09-17'

In [123]:
df_coin_day.index = df_coin_day.index.tz_localize(None)
today = pd.to_datetime(datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=0))
today_open_price = round(df_coin_day[df_coin_day.index == today]['Open'][0],2)
today_open_price

30309.4

In [102]:
fiftytwo_weeks = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(weeks=52))
high_fiftytwo_weeks=df_coin[df_coin.index >= fiftytwo_weeks]['High'].max()
low_fiftytwo_weeks=df_coin[df_coin.index >= fiftytwo_weeks]['Low'].min()

In [121]:
# last volume 
yesterday = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=1))
weekly_yesterday = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=7))
before_volume = df_coin[df_coin.index == weekly_yesterday]['Volume'][0]
last_volume = df_coin[df_coin.index == yesterday]['Volume'][0]
get_percentage_img(last_volume, before_volume)

In [95]:
today_high = df_coin_day[df_coin_day.index >= today]['High'].max()
today_low = df_coin_day[df_coin_day.index >= today]['Low'].min()
price_range = str(round(today_high, 2))+'-'+str(round(today_low, 2))
price_range

'30590.59-30063.73'